# Representación de Texto con Ingeniería de Características

## Explorando Modelos Estadísticos Tradicionales

En este notebook se explorarán las siguientes técnicas de ingeniería de características:

- Modelo de Bolsa de Palabras (TF)
- Modelo de Bolsa de N-gramas
- Modelo TF-IDF
- Características de Similitud

## Preparar un Corpus de Ejemplo

Vamos a usar un pequeño corpus de documentos de ejemplo sobre el cual realizaremos la mayoría de los análisis. Un **corpus** es una colección de documentos de texto, normalmente relacionados con uno o más temas o dominios.

In [15]:
import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 200

corpus = ['The sky is blue and beautiful.',
          'Love this blue and beautiful sky!',
          'The quick brown fox jumps over the lazy dog.',
          "A king's breakfast has sausages, ham, bacon, eggs, toast and beans",
          'I love green eggs, ham, sausages and bacon!',
          'The brown fox is quick and the blue dog is lazy!',
          'The sky is very blue and the sky is very beautiful today',
          'The dog is lazy but the brown fox is quick!'    
]
labels = ['weather', 'weather', 'animals', 'food', 'food', 'animals', 'weather', 'animals']

corpus = np.array(corpus)
corpus_df = pd.DataFrame({'Document': corpus, 
                          'Category': labels})
corpus_df = corpus_df[['Document', 'Category']]
corpus_df

,Document,Category
0,The sky is blue and beautiful.,weather
1,Love this blue and beautiful sky!,weather
2,The quick brown fox jumps over the lazy dog.,animals
3,"A king's breakfast has sausages, ham, bacon, eggs, toast and beans",food
4,"I love green eggs, ham, sausages and bacon!",food
5,The brown fox is quick and the blue dog is lazy!,animals
6,The sky is very blue and the sky is very beautiful today,weather
7,The dog is lazy but the brown fox is quick!,animals


Los documentos de ejemplo que pertenecen a diferentes categorías para nuestro corpus de prueba. Antes de hablar sobre ingeniería de características, necesitamos realizar un **preprocesamiento** o **limpieza de datos **para eliminar caracteres, símbolos y tokens innecesarios.

## Preprocesamiento de Texto Simple

Como el enfoque de esta unidad es la ingeniería de características, construiremos un preprocesador de texto sencillo. Este se encargará de eliminar caracteres especiales, espacios extra, dígitos, palabras vacías (stopwords), y transformará todo el texto a minúsculas.

In [16]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cmillan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/cmillan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
import nltk
import re

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(corpus)
norm_corpus

array(['sky blue beautiful', 'love blue beautiful sky',
       'quick brown fox jumps lazy dog',
       'kings breakfast sausages ham bacon eggs toast beans',
       'love green eggs ham sausages bacon',
       'brown fox quick blue dog lazy', 'sky blue sky beautiful today',
       'dog lazy brown fox quick'], dtype='<U51')

## Modelo de Bolsa de Palabras - TF

Es uno de los modelos más simples para representar texto no estructurado como vectores numéricos. Cada documento es representado como una bolsa de palabras, ignorando el orden y la gramática. Cada dimensión del vector representa una palabra del vocabulario y su valor puede ser la frecuencia, presencia o peso.

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]])

Nuestros documentos han sido convertidos en vectores numéricos, de modo que cada documento está representado por un vector (fila) en la matriz de características mostrada arriba. El siguiente código ayudará a representar esto en un formato más fácil de entender

In [19]:
# get all unique words in the corpus
vocab = cv.get_feature_names_out()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

,bacon,beans,beautiful,blue,breakfast,brown,dog,eggs,fox,green,ham,jumps,kings,lazy,love,quick,sausages,sky,toast,today
0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,0,0,0,0,0,1,1,0,1,0,0,1,0,1,0,1,0,0,0,0
3,1,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0
4,1,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0
5,0,0,0,1,0,1,1,0,1,0,0,0,0,1,0,1,0,0,0,0
6,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1
7,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,1,0,0,0,0


En cada columna o dimensión de los vectores de características representa una palabra del corpus, y cada fila representa uno de nuestros documentos. El valor en cualquier celda representa la cantidad de veces que esa palabra (representada por la columna) aparece en el documento específico (representado por la fila). Por lo tanto, si un corpus de documentos contiene N palabras únicas en total, tendríamos un vector de N dimensiones para cada uno de los documentos.

## Modelo de Bolsa de N-gramas

Este modelo generaliza el de bolsa de palabras incluyendo secuencias de N palabras consecutivas (N-gramas). Esto permite capturar información contextual como frases comunes y patrones de lenguaje más ricos que el modelo de palabras individuales.

In [20]:
# you can set the n-gram range to 1,2 to get unigrams as well as bigrams
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)

bv_matrix = bv_matrix.toarray()
vocab = bv.get_feature_names_out()
pd.DataFrame(bv_matrix, columns=vocab)

,bacon eggs,beautiful sky,beautiful today,blue beautiful,blue dog,blue sky,breakfast sausages,brown fox,dog lazy,eggs ham,...,lazy dog,love blue,love green,quick blue,quick brown,sausages bacon,sausages ham,sky beautiful,sky blue,toast beans
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
5,0,0,0,0,1,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0
6,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
7,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


Esto nos proporciona vectores de características para nuestros documentos, donde cada característica consiste en un bi-grama que representa una secuencia de dos palabras, y los valores indican cuántas veces estuvo presente dicho bi-grama en nuestros documentos.

## Modelo TF-IDF

La Frecuencia Inversa de Documento (IDF) es un factor de pesado que mide lo inusual de un término en el corpus. Es utilizado frecuentemente para reducir la influencia de términos comunes para el análisis de datos o del aprendizaje automático. Para explicarlo, se define document frecuency ($df$) de un termino $t$. Dado un corpus (conjunto de documentos) $C$. El $df(t)$ es simplemente el número de documentos $d$ en $C$ que contienen el término $t$. 

$df(t) = |\{d \in C | t \in d\}|$

Los términos que aparecen en muchos documentos tienen un alto $df$. Basado en esto se puede definir el _inverse document frequency idf(t)_ como:

$idf(t) = \log \big( \frac{|C|}{df(t)} \big)$

El logaritmo es utilizado para la escala sublineal. De otro modo, palabras raras podrían tomar valores IDF extremadamente altos. Nótese que $idf(t)=0$ para términos que aparecen en todos los documentos, es decir, $df(t) = |C|$. Para no ignorar completamente estos términos, algunas librerías agregan una constante al termino completo. Se agregar el termino 0.1, el cual es aproximadamente el valor de los tokens que aparecen en el 90% de los documentos $(\log(1/0.9))$.

Para el pesado de un término $t$ en un conjunto de documentos $D \subset C$, se calcula el puntaje TF-IDF como el producto de $tf(t,D)$ y el IDF del término $t$:

$tfidf(t, D) = tf(t, D) \cdot idf(t)$

Esta puntuación produce altos valores para términos que aparecen frecuentemente en los documentos seleccionados $D$ pero raramente en otros documentos del corpus.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names_out()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

,bacon,beans,beautiful,blue,breakfast,brown,dog,eggs,fox,green,ham,jumps,kings,lazy,love,quick,sausages,sky,toast,today
0,0.00,0.00,0.60,0.53,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.60,0.00,0.0
1,0.00,0.00,0.49,0.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.57,0.00,0.00,0.49,0.00,0.0
2,0.00,0.00,0.00,0.00,0.00,0.38,0.38,0.00,0.38,0.00,0.00,0.53,0.00,0.38,0.00,0.38,0.00,0.00,0.00,0.0
3,0.32,0.38,0.00,0.00,0.38,0.00,0.00,0.32,0.00,0.00,0.32,0.00,0.38,0.00,0.00,0.00,0.32,0.00,0.38,0.0
4,0.39,0.00,0.00,0.00,0.00,0.00,0.00,0.39,0.00,0.47,0.39,0.00,0.00,0.00,0.39,0.00,0.39,0.00,0.00,0.0
5,0.00,0.00,0.00,0.37,0.00,0.42,0.42,0.00,0.42,0.00,0.00,0.00,0.00,0.42,0.00,0.42,0.00,0.00,0.00,0.0
6,0.00,0.00,0.36,0.32,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.72,0.00,0.5
7,0.00,0.00,0.00,0.00,0.00,0.45,0.45,0.00,0.45,0.00,0.00,0.00,0.00,0.45,0.00,0.45,0.00,0.00,0.00,0.0


Los vectores de características basados en TF-IDF para cada uno de nuestros documentos de texto muestran valores escalados y normalizados en comparación con los valores sin procesar del modelo de Bolsa de Palabras.

# Similitud entre Documentos

La similitud entre documentos es el proceso de utilizar una métrica basada en distancia o similitud que permita identificar qué tan similar es un documento de texto con respecto a otro(s), basándose en características extraídas de los documentos, como la bolsa de palabras o TF-IDF.

Como puedes ver, podemos construir sobre las características basadas en TF-IDF que diseñamos en la sección anterior y utilizarlas para generar nuevas características que pueden ser útiles en áreas como motores de búsqueda, agrupamiento de documentos y recuperación de información, aprovechando estas características basadas en similitud.

La similitud por pares entre documentos en un corpus implica calcular la similitud entre cada par de documentos. Por lo tanto, si tienes C documentos en un corpus, terminarás con una matriz de C x C en la que cada fila y cada columna representan el puntaje de similitud para un par de documentos, que corresponden a los índices de la fila y la columna, respectivamente.

Existen diversas métricas de similitud y distancia que se utilizan para calcular la similitud entre documentos. Estas incluyen:
	•	Distancia o similitud del coseno
	•	Distancia euclidiana
	•	Distancia de Manhattan
	•	Similitud BM25
	•	Distancia de Jaccard
	•	Entre otras.

En nuestro análisis, utilizaremos quizás la métrica de similitud más popular y ampliamente usada: la similitud del coseno, y compararemos la similitud por pares entre documentos usando sus vectores de características TF-IDF.

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

,0,1,2,3,4,5,6,7
0,1.000000,0.820599,0.000000,0.000000,0.000000,0.192353,0.817246,0.000000
1,0.820599,1.000000,0.000000,0.000000,0.225489,0.157845,0.670631,0.000000
2,0.000000,0.000000,1.000000,0.000000,0.000000,0.791821,0.000000,0.850516
3,0.000000,0.000000,0.000000,1.000000,0.506866,0.000000,0.000000,0.000000
4,0.000000,0.225489,0.000000,0.506866,1.000000,0.000000,0.000000,0.000000
5,0.192353,0.157845,0.791821,0.000000,0.000000,1.000000,0.115488,0.930989
6,0.817246,0.670631,0.000000,0.000000,0.000000,0.115488,1.000000,0.000000
7,0.000000,0.000000,0.850516,0.000000,0.000000,0.930989,0.000000,1.000000


La similitud del coseno básicamente nos proporciona una métrica que representa el coseno del ángulo entre las representaciones vectoriales de características de dos documentos de texto. Cuanto menor es el ángulo entre los documentos, más cercanos y similares son entre sí, como se muestra en la siguiente figura.

![fig](https://media.datacamp.com/cms/google/ad_4nxci_8xi1a_ohgyiu2xrqejfw11nvwqr9gxfdw-jl7iz9wdciujm99dyb2qan6rrs9tah84pe-ex3ujuikadeyrooftcgdvwsonyjavrj73bgrssehlkilxftpzzmvjzqk2liqo5ehna1w8fvgkchdcbbdpk.png)


Al observar de cerca la matriz de similitud, se puede ver claramente que los documentos (0, 1 y 6), así como (2, 5 y 7), son muy similares entre sí. Los documentos 3 y 4 son ligeramente similares entre ellos, aunque la magnitud de similitud no es muy fuerte; sin embargo, sigue siendo más alta que la de otros documentos. Esto indica que esos documentos similares comparten algunas características en común.

Este es un ejemplo perfecto de agrupamiento o clustering, que puede resolverse mediante aprendizaje no supervisado, especialmente cuando se trabaja con grandes corpus de millones de documentos de texto.

### Bonus: Agrupamiento usando características de similitud entre documentos

Usaremos un método de agrupamiento muy popular basado en particiones: K-means clustering, para agrupar o clasificar estos documentos en función de sus representaciones de características basadas en similitud.

En el agrupamiento K-means, tenemos un parámetro de entrada k, que especifica el número de clústeres que se generarán utilizando las características de los documentos. Este método de agrupamiento se basa en centroides, y busca agrupar los documentos en clústeres con varianza similar.

El algoritmo intenta crear estos clústeres minimizando la suma de los errores cuadrados dentro del clúster, una medida que también se conoce como inercia.

In [29]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=3, random_state=0)
km.fit_transform(similarity_matrix)


array([[1.82791431, 2.05405026, 0.14779036],
       [1.66782816, 2.0292346 , 0.28281211],
       [1.87039875, 0.20458759, 2.06995657],
       [0.36647252, 1.92987836, 1.83327747],
       [0.36647252, 1.93689125, 1.72137257],
       [1.91870105, 0.22340705, 1.909644  ],
       [1.7685556 , 2.0338377 , 0.25641253],
       [1.93342988, 0.12299811, 2.11690202]])

We can see from the above output that our documents were correctly assigned to the right clusters!

In [30]:
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
pd.concat([corpus_df, cluster_labels], axis=1)

,Document,Category,ClusterLabel
0,The sky is blue and beautiful.,weather,2
1,Love this blue and beautiful sky!,weather,2
2,The quick brown fox jumps over the lazy dog.,animals,1
3,"A king's breakfast has sausages, ham, bacon, eggs, toast and beans",food,0
4,"I love green eggs, ham, sausages and bacon!",food,0
5,The brown fox is quick and the blue dog is lazy!,animals,1
6,The sky is very blue and the sky is very beautiful today,weather,2
7,The dog is lazy but the brown fox is quick!,animals,1


Podemos ver en la salida anterior que nuestros documentos fueron asignados correctamente a los clústeres adecuados.